In [0]:
# import 
from keras.datasets import cifar100
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Input, Dense, Flatten, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical

import argparse
import numpy as np

In [0]:
# model作成 CNNByChainerと同じ層構成にする
def create_CNN_model(input_shape=(32, 32, 3), class_num=100):
    input = Input((32, 32, 3))
    kernel_size = (3, 3)
    max_pool_size = (2, 2)
    # 畳み込み層の実装
    # 1層目
    cnn = Conv2D(64, kernel_size=kernel_size, padding='same', strides=(1, 1), activation='relu', input_shape=(32, 32, 3))(input)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Conv2D(128, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Dropout(0.25)(cnn)
    cnn = Conv2D(256, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Conv2D(512, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Dropout(0.25)(cnn)
    cnn = Conv2D(512, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)

    # 入力を平滑化する層（いわゆるデータをフラット化する層、例えば4次元配列を1次元配列に変換するなど）
    fc = Flatten()(cnn)
    # denseは全結合層
    fc = Dense(1000, activation='relu')(fc)
    fc = Dropout(0.5)(fc)
    softmax = Dense(100, activation='softmax')(fc)
    model = Model(input=input, output=softmax)
    
    return model
 
def model_family_cnn(input_shape=(32, 32, 3), num_classes=100):
    input_layer = Input(shape=input_shape)

    # Block 1
    conv1_1 = Conv2D(64, (3, 3),name='conv1_1', activation='relu', padding='same')(input_layer)
    conv1_2 = Conv2D(64, (3, 3),name='conv1_2', activation='relu', padding='same')(conv1_1)
    bn1 = BatchNormalization(axis=3)(conv1_2)
    pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)
    drop1 = Dropout(0.5)(pool1)

    # Block 2
    conv2_1 = Conv2D(128, (3, 3),name='conv2_1', activation='relu', padding='same')(drop1)
    conv2_2 = Conv2D(128, (3, 3),name='conv2_2', activation='relu', padding='same')(conv2_1)
    bn2 = BatchNormalization(axis=3)(conv2_2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)
    drop2 = Dropout(0.5)(pool2)

    # Block 3
    conv3_1 = Conv2D(256, (3, 3),name='conv3_1', activation='relu', padding='same')(drop2)
    conv3_2 = Conv2D(256, (3, 3),name='conv3_2', activation='relu', padding='same')(conv3_1)
    conv3_3 = Conv2D(256, (3, 3),name='conv3_3', activation='relu', padding='same')(conv3_2)
    conv3_4 = Conv2D(256, (3, 3),name='conv3_4', activation='relu', padding='same')(conv3_3)
    bn3 = BatchNormalization(axis=3)(conv3_4)
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)
    drop3 = Dropout(0.5)(pool3)

    # Block 4
    conv4_1 = Conv2D(512, (3, 3),name='conv4_1', activation='relu', padding='same')(drop3)
    conv4_2 = Conv2D(512, (3, 3),name='conv4_2', activation='relu', padding='same')(conv4_1)
    conv4_3 = Conv2D(512, (3, 3),name='conv4_3', activation='relu', padding='same')(conv4_2)
    conv4_4 = Conv2D(512, (3, 3),name='conv4_4', activation='relu', padding='same')(conv4_3)
    bn4 = BatchNormalization(axis=3)(conv4_4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(bn4)
    drop4 = Dropout(0.5)(pool4)

    # Block 5
    conv5_1 = Conv2D(512, (3, 3),name='conv5_1', activation='relu', padding='same')(drop4)
    conv5_2 = Conv2D(512, (3, 3),name='conv5_2', activation='relu', padding='same')(conv5_1)
    conv5_3 = Conv2D(512, (3, 3),name='conv5_3', activation='relu', padding='same')(conv5_2)
    conv5_4 = Conv2D(512, (3, 3),name='conv5_4', activation='relu', padding='same')(conv5_3)
    bn5 = BatchNormalization(axis=3)(conv5_4)
    pool5 = MaxPooling2D(pool_size=(2, 2))(bn5)
    drop5 = Dropout(0.5)(pool5)
    
    x = Flatten()(drop5)
    x = Dense(4096)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    x = Activation('softmax')(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

def train():

    model = create_CNN_model()
    # cifar10のデータ取得
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    x_train1, x_valid, y_train1, y_valid = train_test_split(x_train, y_train, test_size=0.2)
    # RGB画像で32×32なので32×32×3にreshapeする
    x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
    x_valid = x_valid.reshape(x_valid.shape[0], 32, 32, 3)
    x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
    
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_valid /= 255
    x_test /= 255

    # one-hot vector形式に変換する
    y_train = to_categorical(y_train, 100)
    y_valid = to_categorical(y_valid, 100)
    y_test = to_categorical(y_test, 100)
    
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(),  metrics=['accuracy'])
    # 学習
    model.fit(x_train, y_train, epochs=40, batch_size=128, verbose=1, validation_data=(x_valid, y_valid))
    # 精度算出
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [19]:
train()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 24s 472us/step - loss: 4.3217 - acc: 0.0329 - val_loss: 3.9655 - val_acc: 0.0682
Epoch 2/40
50000/50000 [==============================] - 22s 433us/step - loss: 3.7820 - acc: 0.1038 - val_loss: 3.4148 - val_acc: 0.1731
Epoch 3/40
50000/50000 [==============================] - 22s 436us/step - loss: 3.4173 - acc: 0.1701 - val_loss: 3.0982 - val_acc: 0.2355
Epoch 4/40
50000/50000 [==============================] - 22s 436us/step - loss: 3.1524 - acc: 0.2203 - val_loss: 2.8311 - val_acc: 0.2883
Epoch 5/40
50000/50000 [==============================] - 22s 435us/step - loss: 2.9238 - acc: 0.2654 - val_loss: 2.5136 - val_acc: 0.3489
Epoch 6/40
50000/50000 [==============================] - 22s 435us/step - loss: 2.7504 - acc: 0.3009 - val_loss: 2.3630 - val_acc: 0.3848
Epoch 7/40
50000/50000 [==============================] - 22s 434us/step - loss: 2.5957 - acc: 0.3320 - val_loss: 2.